In [28]:
import iris
import sys
import importlib
import numpy as np
from random import random, uniform
from copy import deepcopy

In [80]:
importlib.reload(iris)

<module 'iris' from 'E:\\REPOS\\int_sys\\swarm\\iris.py'>

In [121]:
class Particle:
    def __init__(self, X_train, y_train, min_X=-10, max_X=10, c1 = 1.49445, c2 = 1.49445, w = 0.729):
        self.min_X = min_X
        self.max_X = max_X
        self.c1 = c1
        self.c2 = c2
        self.w = w
        
        nn = iris.IrisNN(gen_randomly=True) # 4-6-3 neural net with randomly generated parameters
        self.position = nn.merge_params()     
        self.init_velocity()
        self.fitness = nn.compute_mse(X_train, y_train)
        self.pbest = deepcopy([self.position, self.fitness])
    
    # t.random Microsoft guy
    def init_velocity(self):
        lo = 0.1 * self.min_X;
        hi = 0.1 * self.max_X;
        self.velocity = np.zeros(len(self.position))
        for i in range(0, len(self.position)):
            self.velocity[i] = (hi - lo) * random() + lo;
     
    def compute_fitness(self, X_train, y_train):
        nn = iris.IrisNN()
        nn.unpack_params(self.position)
        self.fitness = nn.compute_mse(X_train, y_train)
        if self.fitness < self.pbest[1]:
            self.pbest = deepcopy([self.position, self.fitness])
            
    def update(self, gbest):
        self.velocity = (self.w * self.velocity) \
                                      + (random() * self.c1 * (self.pbest[0] - self.position))\
                                      + (random() * self.c2 * (gbest[0] - self.position))
            
        self.position += self.velocity
        self.position[self.position < -10.0] = -10.0
        self.position[self.position > 10.0] = 10.0
        

In [125]:
class PSO:
    def __init__(self, swarm_size=12, epochs=700, use_subset=False, subset=[24, 6]):
        if swarm_size < 1 or epochs < 0 or len(subset) != 2:
            print('Invalid input.Exiting..')
            sys.exit(-1)
        self.swarm_size = swarm_size
        self.epochs = epochs
        self.df = iris.IrisDF()
        if use_subset:
            self.df.subset(subset[0], subset[1])
        self.gen_random_swarm()
    
    def gen_random_swarm(self):
        self.swarm = []
        self.swarm.append(Particle(self.df.X_train, self.df.y_train))
        self.gbest = swarm[0].pbest
        for i in range(1, self.swarm_size):
            p = Particle(self.df.X_train, self.df.y_train)
            if p.pbest[1] < self.gbest[1]:
                self.gbest = p.pbest
            self.swarm.append(p)
        
    def optimize(self):
        for i in range(0, self.epochs):
            for p in self.swarm:
                # update velocity and position
                p.update(self.gbest)
                # update fitness and eventualy pbest
                p.compute_fitness(self.df.X_train, self.df.y_train)
                # update gbest
                if(i % 100 == 0): print(p.pbest[1])
                if p.pbest[1] < self.gbest[1]:
                    self.gbest = p.pbest
            if i % 100 == 0:
                print('Epoch {} with gbest mse of {}.'.format(i, self.gbest[1]))
                    
    def show_accuracy(self):
        nn = iris.IrisNN()
        nn.unpack_params(gbest[0])
        acc = nn.compute_accuracy(self.df.X_test, self.df.y_test)
        print("Maximal achieved accuracy is {}".format(acc))
    

In [134]:
pso = PSO()
pso.optimize()

0.766914253531
0.675978169355
0.768810620326
0.786268846314
0.690645909353
0.739510263878
0.726256820822
0.662206229506
0.722030305624
0.729272663291
0.672150870229
0.681309471239
Epoch 0 with gbest mse of 0.6622062295063361.
0.307227004082
0.307218424491
0.308177980899
0.309061849904
0.307336080573
0.310220665836
0.307312997457
0.307347777512
0.307629062152
0.307241665822
0.307235764699
0.307553930916
Epoch 100 with gbest mse of 0.307218424490515.
0.300413978052
0.300646604737
0.300592355075
0.300475250923
0.300671327344
0.301199342135
0.302964800873
0.30043305131
0.300473893165
0.300658580984
0.300589101894
0.300379989825
Epoch 200 with gbest mse of 0.30037998982467795.
0.170305658628
0.170261389032
0.170246540537
0.170273831276
0.17029450259
0.170263201093
0.170243920445
0.170280123167
0.170216326574
0.170267781638
0.170197111003
0.170228982754
Epoch 300 with gbest mse of 0.17019711100308396.
0.133278133351
0.133257238097
0.133252348537
0.133251099444
0.133249315591
0.133262952321
0

In [135]:
pso.show_accuracy()

Maximal achieved accuracy is 0.24444444444444444


In [136]:
nn = iris.IrisNN()
nn.unpack_params(pso.gbest[0])
acc = nn.compute_accuracy(pso.df.X_train, pso.df.y_train)
print("Maximal achieved accuracy is {}".format(acc))

Maximal achieved accuracy is 0.9619047619047619
